In [ ]:
!pip -q install pandas numpy scikit-learn tensorflow matplotlib joblib streamlit

In [ ]:
from google.colab import files
up = files.upload()

In [ ]:
DATA_PATH = 'Crop_recommendation.csv'

ARTIFACTS_DIR = Path("/content/artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
!pip install -q scikit-learn joblib gradio==4.44.0

import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, utils
tf.__version__

In [ ]:
df = pd.read_csv(DATA_PATH).drop_duplicates().dropna()

FEATURES = ["N", "P", "K", "temperature", "humidity", "ph", "rainfall"]
TARGET = "label"

X = df[FEATURES].values.astype("float32")
y_text = df[TARGET].values

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y_text)
num_classes = len(le.classes_)
y_cat = utils.to_categorical(y, num_classes=num_classes)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_cat, test_size=0.30, random_state=42, stratify=y_cat
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

In [ ]:
y_train_int = np.argmax(y_train, axis=1)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.arange(num_classes),
    y=y_train_int
)
class_weights = {i: w for i, w in enumerate(class_weights)}

In [ ]:
def build_model(input_dim, num_classes):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.25),

        layers.Dense(64, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.25),

        layers.Dense(32, activation="relu"),
        layers.BatchNormalization(),

        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_model(X_train_s.shape[1], num_classes)

In [ ]:
es = callbacks.EarlyStopping(monitor="val_accuracy", patience=15, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-5)
ckpt_path = ARTIFACTS_DIR / "best_model.keras"
mc = callbacks.ModelCheckpoint(filepath=str(ckpt_path), monitor="val_accuracy", save_best_only=True)

# ---- Train
history = model.fit(
    X_train_s, y_train,
    validation_data=(X_val_s, y_val),
    epochs=200,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[es, rlr, mc],
    verbose=1
)

In [ ]:
best_model = tf.keras.models.load_model(ckpt_path)
y_pred_probs = best_model.predict(X_test_s)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))
print(f"\nTest Accuracy: {(y_true == y_pred).mean():.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
joblib.dump(scaler, ARTIFACTS_DIR / "scaler.joblib")
with open(ARTIFACTS_DIR / "label_encoder_classes.json", "w") as f:
    json.dump(le.classes_.tolist(), f)
print("\nSaved:")
print(f"- Model: {ckpt_path}")
print(f"- Scaler: {ARTIFACTS_DIR / 'scaler.joblib'}")
print(f"- Label classes: {ARTIFACTS_DIR / 'label_encoder_classes.json'}")

In [ ]:
import numpy as np, json, joblib, tensorflow as tf

best_model = tf.keras.models.load_model("/content/artifacts/best_model.keras")
scaler = joblib.load("/content/artifacts/scaler.joblib")
classes = json.loads(open("/content/artifacts/label_encoder_classes.json").read())

FEATURES = ["N","P","K","temperature","humidity","ph","rainfall"]

sample = {
    "N":90, "P":42, "K":43,
    "temperature":26.5, "humidity":80.0, "ph":6.5, "rainfall":200.0
}
x = np.array([[sample[f] for f in FEATURES]], dtype="float32")
x_s = scaler.transform(x)
probs = best_model.predict(x_s)[0]
pred = classes[int(np.argmax(probs))]
pred, float(np.max(probs))
